<a href="https://colab.research.google.com/github/Geriys/VKirillV/blob/main/HW_3_ML11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
%matplotlib inline

from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [16]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [17]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [20]:
### Ваш код ###
from sklearn.model_selection import train_test_split
y = data["MEDV"]
X = data.drop(columns=["MEDV"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 10, shuffle = True)
#print(f"Классы в y_train:\n{y_train}\n")
#print(f"Классы в y_test:\n{y_test}")

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [21]:

from sklearn.linear_model import LinearRegression # Линейная регресси
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error

### Ваш код ###
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
# lr_y_pred = lr_model.predict(X_test)

ridge_model = Ridge()
ridge_model.fit(X_train, y_train)
# ridge_y_pred = ridge_model.predict(X_test)

lasso_model = Lasso()
lasso_model.fit(X_train, y_train)
# lasso_y_pred = lasso_model.predict(X_test)

# https://scikit-learn.ru/3-1-cross-validation-evaluating-estimator-performance/
# https://datafinder.ru/products/ocenka-r2-v-mashinnom-obuchenii#:~:text=%D0%9E%D1%86%D0%B5%D0%BD%D0%BA%D0%B0%20R2%20%E2%80%93%20%D0%BE%D1%87%D0%B5%D0%BD%D1%8C%20%D0%B2%D0%B0%D0%B6%D0%BD%D1%8B%D0%B9%20%D0%BF%D0%BE%D0%BA%D0%B0%D0%B7%D0%B0%D1%82%D0%B5%D0%BB%D1%8C,%D0%B2%20%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7%D0%B0%D1%85%2C%20%D0%BE%D0%B1%D1%8A%D1%8F%D1%81%D0%BD%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%BD%D0%B0%D0%B1%D0%BE%D1%80%D0%BE%D0%BC%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85.
# Calculation of R2 Score
r2_lr = lr_model.score(X_test, y_test) #cross_val_score(lr_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()
r2_ridge = ridge_model.score(X_test, y_test) #cross_val_score(ridge_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()
r2_lasso = lasso_model.score(X_test, y_test) #cross_val_score(lasso_model, X_test, y_test, cv=KFold(n_splits=3), scoring="r2").mean()

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]
print(results_regression)

   model   task        R2
0     LR  task2  0.670642
1  Ridge  task2  0.665291
2  Lasso  task2  0.534676


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [23]:

# https://medium.com/nuances-of-programming/%D0%BB%D0%B0%D1%81%D1%81%D0%BE-%D0%B8-%D1%80%D0%B8%D0%B4%D0%B6-%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D0%B8-%D0%B8%D0%BD%D1%82%D1%83%D0%B8%D1%82%D0%B8%D0%B2%D0%BD%D0%BE%D0%B5-%D1%81%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5-a542dd761f62
### Ваш код ###
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV

# Defining grid of candidate alpha values (powers of 10, from 0.00001 to 1000000)
param_grid = {"alpha": 10.0 ** np.arange(-5, 6)}

# Initializing Ridge and GridSearchCV estimators
ridge = Ridge()
print(ridge.get_params().keys())
grid_search = GridSearchCV(estimator=ridge, param_grid=param_grid)
# Fitting grid search object
grid_search.fit(X_train, y_train)
r2_ridge_grid_search = grid_search.score(X_test, y_test)

ridge_reg_cv = RidgeCV(alphas=param_grid['alpha'], cv=5)
ridge_reg_cv.fit(X_train, y_train)
r2_ridge_cv = ridge_reg_cv.score(X_test, y_test)

# Initializing Lasso and GridSearchCV estimators
lasso = Lasso()
grid_search_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid)
# Fitting grid search object
grid_search_lasso.fit(X_train, y_train)
r2_lasso_grid_search = grid_search_lasso.score(X_test, y_test)

lasso_reg_cv = LassoCV(alphas=param_grid['alpha'], cv=5)
lasso_reg_cv.fit(X_train, y_train)

r2_lasso_cv = lasso_reg_cv.score(X_test, y_test)

results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
print(results_regression)

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'positive', 'random_state', 'solver', 'tol'])
                model   task        R2
0                  LR  task2  0.670642
1               Ridge  task2  0.665291
2               Lasso  task2  0.534676
3  Ridge_GridSearchCV  task3  0.670002
4             RidgeCV  task3  0.670002
5  Lasso_GridSearchCV  task3  0.670360
6             LassoCV  task3  0.670360


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [24]:
### Ваш код ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

estimators = [('scaler', StandardScaler()), ('model', Ridge())]
pipe = Pipeline(estimators)
r2_ridge_standart_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', MinMaxScaler()), ('model', Ridge())]
pipe = Pipeline(estimators)
r2_ridge_min_max_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', StandardScaler()), ('model', Lasso())]
pipe = Pipeline(estimators)
r2_lasso_standart_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)

estimators = [('scaler', MinMaxScaler()), ('model', Lasso())]
pipe = Pipeline(estimators)
r2_lasso_min_max_scaler = pipe.fit(X_train, y_train).score(X_test, y_test)
results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]
print(results_regression)

                   model   task        R2
0                     LR  task2  0.670642
1                  Ridge  task2  0.665291
2                  Lasso  task2  0.534676
3     Ridge_GridSearchCV  task3  0.670002
4                RidgeCV  task3  0.670002
5     Lasso_GridSearchCV  task3  0.670360
6                LassoCV  task3  0.670360
7   Ridge_StandardScaler  task4  0.670844
8     Ridge_MinMaxScaler  task4  0.666259
9   Lasso_StandardScaler  task4  0.608150
10    Lasso_MinMaxScaler  task4  0.094319


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [25]:
### Ваш код ###

pipe = Pipeline([('scaler', StandardScaler()), ('model', Ridge())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_standart_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', Ridge())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_min_max_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('model', Lasso())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_standart_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('model', Lasso())])
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_min_max_scaler_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]
print(results_regression)

                      model   task        R2
0                        LR  task2  0.670642
1                     Ridge  task2  0.665291
2                     Lasso  task2  0.534676
3        Ridge_GridSearchCV  task3  0.670002
4                   RidgeCV  task3  0.670002
5        Lasso_GridSearchCV  task3  0.670360
6                   LassoCV  task3  0.670360
7      Ridge_StandardScaler  task4  0.670844
8        Ridge_MinMaxScaler  task4  0.666259
9      Lasso_StandardScaler  task4  0.608150
10       Lasso_MinMaxScaler  task4  0.094319
11  Ridge_StandardScaler_CV  task5  0.671640
12    Ridge_MinMaxScaler_CV  task5  0.670629
13  Lasso_StandardScaler_CV  task5  0.671214
14    Lasso_MinMaxScaler_CV  task5  0.671900


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [26]:
### Ваш код ###
from sklearn.preprocessing import PolynomialFeatures

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
r2_ridge_standart_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
r2_ridge_min_max_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
r2_lasso_standart_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
r2_lasso_min_max_scaler_poly = pipe.fit(X_train, y_train).score(X_test, y_test)


results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
print(results_regression)

                        model   task        R2
0                          LR  task2  0.670642
1                       Ridge  task2  0.665291
2                       Lasso  task2  0.534676
3          Ridge_GridSearchCV  task3  0.670002
4                     RidgeCV  task3  0.670002
5          Lasso_GridSearchCV  task3  0.670360
6                     LassoCV  task3  0.670360
7        Ridge_StandardScaler  task4  0.670844
8          Ridge_MinMaxScaler  task4  0.666259
9        Lasso_StandardScaler  task4  0.608150
10         Lasso_MinMaxScaler  task4  0.094319
11    Ridge_StandardScaler_CV  task5  0.671640
12      Ridge_MinMaxScaler_CV  task5  0.670629
13    Lasso_StandardScaler_CV  task5  0.671214
14      Lasso_MinMaxScaler_CV  task5  0.671900
15  Ridge_StandardScaler_Poly  task6  0.877763
16    Ridge_MinMaxScaler_Poly  task6  0.825928
17  Lasso_StandardScaler_Poly  task6  0.748369
18    Lasso_MinMaxScaler_Poly  task6  0.089644


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [27]:
### Ваш код ###

params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_standart_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Ridge())])
grid_pipeline = GridSearchCV(pipe, params)
r2_ridge_min_max_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_standart_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

pipe = Pipeline([('scaler', MinMaxScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_min_max_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)


results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]
print(results_regression)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.495e+02, tolerance: 2.514e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.146e+02, tolerance: 2.574e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.237e+02, tolerance: 2.353e

                           model   task        R2
0                             LR  task2  0.670642
1                          Ridge  task2  0.665291
2                          Lasso  task2  0.534676
3             Ridge_GridSearchCV  task3  0.670002
4                        RidgeCV  task3  0.670002
5             Lasso_GridSearchCV  task3  0.670360
6                        LassoCV  task3  0.670360
7           Ridge_StandardScaler  task4  0.670844
8             Ridge_MinMaxScaler  task4  0.666259
9           Lasso_StandardScaler  task4  0.608150
10            Lasso_MinMaxScaler  task4  0.094319
11       Ridge_StandardScaler_CV  task5  0.671640
12         Ridge_MinMaxScaler_CV  task5  0.670629
13       Lasso_StandardScaler_CV  task5  0.671214
14         Lasso_MinMaxScaler_CV  task5  0.671900
15     Ridge_StandardScaler_Poly  task6  0.877763
16       Ridge_MinMaxScaler_Poly  task6  0.825928
17     Lasso_StandardScaler_Poly  task6  0.748369
18       Lasso_MinMaxScaler_Poly  task6  0.089644


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.853e+01, tolerance: 3.109e+00
  model = cd_fast.enet_coordinate_descent(


8. Подберите наилучшую модель (используйте Pipeline, GridSearchCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [ ]:
### Ваш код ###
params = { 'model__alpha': 10.0 ** np.arange(-5, 6) }


pipe = Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures()), ('model', Lasso())])
grid_pipeline = GridSearchCV(pipe, params)
r2_lasso_standart_scaler_poly_cv = grid_pipeline.fit(X_train, y_train).score(X_test, y_test)

best_params = {cv: 'GridSearchCV', scaler: 'MinMaxScaler', model: 'Ridge(L2)', best_aplha: grid_pipeline.best_params_, }


print('Параметры лучшей модели:\n', best_params)
r2_best_model = 0
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

In [30]:
results_regression

,model,task,R2
0,LR,task2,0.670642
1,Ridge,task2,0.665291
2,Lasso,task2,0.534676
3,Ridge_GridSearchCV,task3,0.670002
4,RidgeCV,task3,0.670002
5,Lasso_GridSearchCV,task3,0.670360
6,LassoCV,task3,0.670360
7,Ridge_StandardScaler,task4,0.670844
8,Ridge_MinMaxScaler,task4,0.666259
9,Lasso_StandardScaler,task4,0.608150


http://archive.ics.uci.edu/ml/datasets/Adult

In [34]:
data = pd.read_csv('adult.csv')
print(data['class'].unique())#.head()

['<=50K' '>50K']


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [36]:
### Ваш код ###
data["class-num"] = np.where(data["class"] == "<=50K", 1, 0)
y = data["class-num"]
X = data.drop(columns=["class", "class-num"])
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 10, shuffle = True)


10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

> Блок с отступами



In [39]:
#Определить самый часто встречающийся класс 1 или 0.
#Облучить модель фитом как обычно // треин тест сплит на отфильтрованой выборке самого частого класса
#на том что получилось найти аккураси и ф1 скор

print(len(data[data["class-num"] == 1]))
print(len(data[data["class-num"] == 0]))
# Самый частый класс - 1

37155
11687


In [38]:

### Ваш код ###
print(type(X_test))
# X_train_most_frequent = X_test[X_test["class-num"] == 1]
# y_train_most_frequent = y_test[y_test["class-num"] == 1]

f1_most_frequent = 0
acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

<class 'pandas.core.frame.DataFrame'>


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:


### Ваш код ###



Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'class', 'class-num'],
      dtype='object')


12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [ ]:
tasks+=1

### Ваш код ###

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification